In [1]:
!pip install wget
!pip install transformers
!pip install ray
!pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import json
import os
import wget
import gzip
import html
import tarfile
import logging
import sys
import csv

import torch
import torch.nn as nn
from transformers import BertModel, AdamW, BertTokenizer

import pandas as pd
import numpy as np

from functools import partial
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
from sklearn.metrics import mean_squared_error
import gensim
import gensim.downloader as api
from gensim import utils

In [3]:
class ProductRanker(nn.Module):
    def __init__(self, l1=128, l2=32):
        super(ProductRanker, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-uncased')

        self.ranker = nn.Sequential(
            nn.Linear(768, l1),
            nn.BatchNorm1d(l1, affine=False),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(l1, l2),
            nn.BatchNorm1d(l2, affine=False),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(l2,1)
        )

        for param in self.bert.parameters():
            param.requires_grad = False

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state_cls = outputs[0][:, 0, :]
        logits = self.ranker(last_hidden_state_cls)
        return logits

In [4]:
#Constants
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
config = {'l1': 128, 'l2': 32, 'lr': 0.0005, 'batch_size': 100}
model = ProductRanker(config["l1"],config["l2"]).to(device)
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=config["lr"], betas=(0.9, 0.999), eps=1e-08, weight_decay=0,
                            amsgrad=False)
cos = torch.nn.CosineSimilarity(dim=0)

if not os.path.exists("./data"):
    os.makedirs("./data")
if not os.path.exists("./outputs"):
    os.makedirs("./outputs")

logging.basicConfig(filename='./outputs/training.log', filemode='w', format='%(asctime)s %(message)s', level=logging.DEBUG, force=True)
logging.getLogger().addHandler(logging.StreamHandler(sys.stdout))
logger = logging.getLogger()

/home/niklas/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:88: UserWarning: HIP initialization: Unexpected error from hipGetDeviceCount(). Did you run some cuda functions before calling NumHipDevices() that might have already set an error? Error 101: hipErrorInvalidDevice (Triggered internally at ../c10/hip/HIPFunctions.cpp:110.)
  return torch._C._cuda_getDeviceCount() > 0
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTrainin

In [12]:
def downloadData(url, path):
    return wget.download(url, out=path)

In [6]:
def loadJson(filename):
    data = []
    with gzip.open(filename, "rt") as f:
        for line in f:
            stripped = line.strip()
            stripped = stripped.replace("\'", "\"")
            try:
                data.append(json.loads(stripped))
            except json.decoder.JSONDecodeError:
                continue 
    return data

In [7]:
def filterJson(data, extrema_dict):
    filtered = []
    for entry in data:
        if not isGoodJsonEntry(entry):
            continue
        
        sales_rank_key = list(entry['salesRank'].keys())[0]
        sales_rank_value = entry['salesRank'][sales_rank_key]
        min = extrema_dict[sales_rank_key][0]
        max = extrema_dict[sales_rank_key][1]

        tmp = {}
        if min == max:
            tmp['salesRank'] = 0.5
        else:
            tmp['salesRank'] = (sales_rank_value - min) / (max - min)
        tmp['description'] = html.unescape(entry['description'])
        filtered.append(tmp)

    return filtered

In [8]:
def getExtremaDict(data):
    extrema_dict = {}
    for entry in data:
        if not isGoodJsonEntry(entry):
            continue

        sales_rank_key = list(entry['salesRank'].keys())[0]
        sales_rank_value = entry['salesRank'][sales_rank_key]
        if sales_rank_key not in extrema_dict:
            extrema_dict[sales_rank_key] = [sales_rank_value, sales_rank_value] 
        else:
            if extrema_dict[sales_rank_key][0] > sales_rank_value:
                extrema_dict[sales_rank_key][0] = sales_rank_value
            elif extrema_dict[sales_rank_key][1] < sales_rank_value:
                extrema_dict[sales_rank_key][1] = sales_rank_value

    return extrema_dict

In [9]:
def isGoodJsonEntry(entry):
    if 'description' not in entry.keys():
        return False
    if len(entry['description']) == 0:
        return False
    if 'salesRank' not in entry.keys():
        return False
    if len(entry['salesRank']) == 0:
        return False
    return True

In [10]:
def safeJson(filtered, filename):
    with open(filename, 'w') as f:
        for object in filtered:
            json.dump(object, f)
            f.write(os.linesep)

In [32]:
def getJSON(data_dir="./data"):
    filename_meta = os.path.join(data_dir, "meta_Electronics.json.gz")
    filename_filtered_meta = os.path.join(data_dir, "filtered_meta_Electronics.tar.gz")
    filename_destination_meta_train = os.path.join(data_dir, "filtered_meta_Electronics_train.json")
    filename_destination_meta_test = os.path.join(data_dir, "filtered_meta_Electronics_test.json")

    url_meta = "http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/meta_Electronics.json.gz"
    
    if os.path.exists(filename_destination_meta_train) and os.path.exists(filename_destination_meta_test):
        return
    
    if not os.path.exists(filename_meta):
        downloadData(url_meta, filename_meta)
    
    data = loadJson(filename_meta)
    extrema_dict = getExtremaDict(data)
    filtered = filterJson(data, extrema_dict)
    train_set, test_set = torch.utils.data.random_split(filtered, [len(filtered)-5625, 5625])
    safeJson(train_set, filename_destination_meta_train)
    safeJson(test_set, filename_destination_meta_test)
    
getJSON()

In [4]:
def loadGoogleWord2Vec():
    return api.load("word2vec-google-news-300")

model_word2vec = loadGoogleWord2Vec()

In [35]:
def averagePollingDocument(model_word2vec, data_dir="./data"):
    path = os.path.join(data_dir, "filtered_meta_Electronics_test.json")
    df = pd.read_json(path, lines=True) 
    
    len_vec = len(model_word2vec["hello"])
    
    documents = dict()
    for index, row in df.iterrows():
        tmp = torch.zeros(len_vec)
        description = utils.simple_preprocess(row['description'])
        for word in description:
            try:
                tmp = tmp.add(torch.tensor(model_word2vec[word]))
            except KeyError:
                continue
        documents[index] = tmp.div(len(description))
        
    return documents

documents = averagePollingDocument(model_word2vec)

NameError: name 'model_word2vec' is not defined

In [19]:
def averagePollingQuery(query, model_word2vec):
    query = utils.simple_preprocess(query)
    len_vec = len(model_word2vec["hello"])
    
    avg_query = torch.zeros(len_vec)
    for word in query:
        try:
            avg_query += avg_query.add(torch.tensor(model_word2vec[word]))
        except KeyError:
            continue
    avg_query = avg_query / len(query)
    return avg_query



[(tensor(0.4788), 8750), (tensor(0.4092), 15011), (tensor(0.4070), 17988)]


In [ ]:
def getMostSimilair(avg_query, documents, x=10):
    avg_query = averagePollingQuery("test test HELLO", model_word2vec)
    cosine_sim = [(cos(avg_query, value), indx) for indx, value in documents.items()]
    cosine_sim.sort(key=lambda y: -y[0].item())
    most_similair = cosine_sim[:x]
    return most_similair

In [12]:
def dataPreparation(data_dir="./data"):
    path = os.path.join(data_dir, "filtered_meta_Electronics_train.json")
    df = pd.read_json(path, lines=True).sample(n = (225))
    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

    tokenized_set = []
    for index, row in df.iterrows():
        tokenized_set.append([tokenizer.encode_plus(row['description'], truncation = True, return_tensors="pt",
                                                    max_length=512, pad_to_max_length=True), row['salesRank']])
    return torch.utils.data.random_split(tokenized_set, [150, 75])

In [13]:
def createDataloader(dataset, config, shuffle=True, test=False):
    labeled_set = []
    if test:
        for i, doc1 in enumerate(dataset):
            labeled_set.append([[doc1[0]], doc1[1]])
    else:
        for i, doc1 in enumerate(dataset):
            for j, doc2 in enumerate(dataset):
                if i != j:
                    label = 0.0
                    if doc1[1] > doc2[1]:
                        label = 1.0
                    if doc1[1] == doc2[1]:
                        label = 0.5

                    labeled_set.append([[doc1[0], doc2[0]], label])
    return torch.utils.data.DataLoader(labeled_set, batch_size=int(config["batch_size"]), num_workers=2, shuffle=shuffle)

In [14]:
def train(model, loss_fn, optimizer, dataloader, epoch, saving=False, tune=False):
    running_loss = 0.0
    epoch_steps = 0

    train_loss = 0.0
    acc_tp_tn = 0
    acc_total = 0
    
    model.train()
    
    for X,y in dataloader:
        input_ids_1 = X[0]['input_ids'].squeeze().to(device)
        attention_mask_1 = X[0]['attention_mask'].squeeze().to(device)
        input_ids_2 = X[1]['input_ids'].squeeze().to(device)
        attention_mask_2 = X[1]['attention_mask'].squeeze().to(device)

        optimizer.zero_grad()
        out1 = model.forward(input_ids_1, attention_mask_1)
        out2 = model.forward(input_ids_2, attention_mask_2)
        diff = (out1 - out2).squeeze()
        diff = torch.sigmoid(diff).to(device)
        loss = loss_fn(diff, y.float().to(device))
        
        for i in range(len(diff)):
            if (diff[i] >= 0.5 and y[i] == 1) or (diff[i] < 0.5 and y[i] == 0):
                acc_tp_tn += 1
            acc_total +=1

        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        
    logger.info("[%d] train loss: %.10f" % (epoch + 1, train_loss / len(dataloader)))
    logging.info("[%d] train accuracy: %.10f" % (epoch + 1, acc_tp_tn / acc_total))

    if saving:
        path = './outputs/model_' + str(epoch+1) + '.pth'
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
        }, path)
        logger.info("Model saved..")

    if tune:
        with tune.checkpoint_dir(epoch) as checkpoint_dir:
            path = os.path.join(checkpoint_dir, "checkpoint")
            torch.save((model.state_dict(), optimizer.state_dict()), path)

    logger.info("Training finished!")


In [15]:
def val(model, loss_fn, dataloader, epoch, tune=False):
        val_loss = 0.0
        acc_tp_tn = 0
        acc_total = 0
        
        model.eval()
        
        for X, y in dataloader:
            with torch.no_grad():
                input_ids_1 = X[0]['input_ids'].squeeze().to(device)
                attention_mask_1 = X[0]['attention_mask'].squeeze().to(device)
                input_ids_2 = X[1]['input_ids'].squeeze().to(device)
                attention_mask_2 = X[1]['attention_mask'].squeeze().to(device)
                out1 = model.forward(input_ids_1, attention_mask_1)
                out2 = model.forward(input_ids_2, attention_mask_2)
                diff = (out1 - out2).squeeze()
                diff = torch.sigmoid(diff)
                loss = loss_fn(diff, y.float().to(device))
                val_loss += loss.cpu().numpy()
                
                for i in range(len(diff)):
                    if (diff[i] >= 0.5 and y[i] == 1) or (diff[i] < 0.5 and y[i] == 0):
                        acc_tp_tn += 1
                    acc_total +=1

        if(tune):
            tune.report(loss=val_loss)

        logger.info("[%d] val loss: %.10f" % (epoch+1, val_loss/len(dataloader)))
        logging.info("[%d] val acc: %.10f" % (epoch + 1, acc_tp_tn/acc_total))
        return 

In [16]:
def train_tune(config, data_dir="./data"):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = ProductRanker(config["l1"],config["l2"]).to(device)
    loss_fn = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=config["lr"], betas=(0.9, 0.999), eps=1e-08, weight_decay=0,
                                amsgrad=False)
    
    train_set, val_set = dataPreparation(data_dir)
    dataloader_train = createDataloader(train_set, config)
    dataloader_val = createDataloader(val_set, config)

    train(model, optimizer, dataloader_train, 0, tune=True)
    val(model, loss_fn, dataloader_val, 0, tune=True)


In [17]:
def tune_hyperparameters():
    data_dir = os.path.abspath("./data")
    config = {
        "l1": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
        "l2": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
        "lr": tune.loguniform(1e-4, 1e-1),
        "batch_size": tune.choice([2, 4, 8, 16])
    }
    scheduler = ASHAScheduler(
        metric="loss",
        mode="min",
        max_t=4,
        grace_period=1,
        reduction_factor=2)
    reporter = CLIReporter(
        # parameter_columns=["l1", "l2", "lr", "batch_size"],
        metric_columns=["loss", "training_iteration"])
    result = tune.run(
        partial(train_tune, data_dir=data_dir),
        resources_per_trial={"cpu": 12, "gpu": 1},
        config=config,
        num_samples=10,
        scheduler=scheduler,
        progress_reporter=reporter)

    best_trial = result.get_best_trial("loss", "min", "last")
    print("Best trial config: {}".format(best_trial.config))
    print("Best trial final validation loss: {}".format(
        best_trial.last_result["loss"]))

In [18]:
def tune_lr():
    data_dir = os.path.abspath("./data")
    config = {
        'l1': 256,
        'l2': 8,
        'lr': tune.loguniform(1e-4, 1e-1),
        'batch_size': 4}
    scheduler = ASHAScheduler(
        metric="loss",
        mode="min",
        max_t=5,
        grace_period=1,
        reduction_factor=2)
    reporter = CLIReporter(
        metric_columns=["loss", "training_iteration"])
    result = tune.run(
        partial(train_tune, data_dir=data_dir),
        resources_per_trial={"cpu": 12, "gpu": 1},
        local_dir="./ray_results",
        config=config,
        num_samples=5,
        scheduler=scheduler,
        progress_reporter=reporter)

    best_trial = result.get_best_trial("loss", "min", "last")
    print("Best trial config: {}".format(best_trial.config))
    print("Best trial final validation loss: {}".format(
        best_trial.last_result["loss"]))
    best_checkpoint = result.get_best_checkpoint(best_trial, metric="loss", mode="min", )

In [19]:
def train_model(epochs, config, from_checkpoint=False, path=None):
    data_dir = os.path.abspath("./data")
    train(epochs, config, data_dir, from_checkpoint, path)

In [33]:
def predict(dataloader,model_filename):
    checkpoint = torch.load(model_filename)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    return_ranks = {}
    number_of_batches = 0
    for X, y in dataloader:
        with torch.no_grad():
            input_ids = X[0]['input_ids'].squeeze().to(device)
            attention_mask = X[0]['attention_mask'].squeeze().to(device)
            out = model.forward(input_ids, attention_mask).squeeze().cpu().numpy()
            for i, result in enumerate(out):
                return_ranks[i+number_of_batches*100] = result       
        number_of_batches += 1       

    return {key: rank for rank, key in enumerate(sorted(return_ranks, key=return_ranks.get, reverse=True), 1)}

In [21]:
def training_loop(epochs, train_bool=True, validate_bool=True, data_dir="./data", saving=False, load=False):
    getJSON()
    train_set, val_set = dataPreparation()
    dataloader_train = createDataloader(train_set, config)
    dataloader_val = createDataloader(val_set, config)
    
    if train_bool or validate_bool:
        for epoch in range(epochs):
            if load:
                current_model_filename = './outputs/model_' + str(epoch+1) + '.pth'
                if os.path.exists(current_model_filename):
                        checkpoint = torch.load(current_model_filename)
                        model.load_state_dict(checkpoint['model_state_dict'])
                        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
                else:
                    logger.info(f"model: {current_model_filename} doesnt exist")

            logger.info(f"Epoch {epoch + 1}\n-------------------------------")
            if train_bool:
                logger.info("Starting training loop...")
                train(model, loss_fn, optimizer, dataloader_train, epoch, saving=saving, tune=False)

            if validate_bool:
                logger.info("Starting validation loop...")
                val(model, loss_fn, dataloader_val, epoch, tune=False)

In [22]:
#training_loop(5, train_bool=False, validate_bool=False, saving=False, load=False)

In [34]:
path = os.path.join('./data', "filtered_meta_Electronics_test.json")
df = pd.read_json(path, lines=True).sample(n = (100))
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

tokenized_set = []
for index, row in df.iterrows():
    tokenized_set.append([tokenizer.encode_plus(row['description'], truncation = True, return_tensors="pt",
                                                max_length=512, pad_to_max_length=True), row['salesRank']])

dataloader = createDataloader(tokenized_set, config, test=True)
print(predict(dataloader, './outputs/model_15.pth'))

Starting new HTTPS connection (1): huggingface.co:443
https://huggingface.co:443 "HEAD /bert-base-uncased/resolve/main/vocab.txt HTTP/1.1" 200 0


/home/christian/.local/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


{66: 1, 87: 2, 27: 3, 0: 4, 46: 5, 99: 6, 18: 7, 68: 8, 69: 9, 53: 10, 3: 11, 5: 12, 43: 13, 84: 14, 11: 15, 31: 16, 20: 17, 81: 18, 45: 19, 17: 20, 59: 21, 50: 22, 52: 23, 72: 24, 6: 25, 38: 26, 60: 27, 78: 28, 24: 29, 35: 30, 64: 31, 33: 32, 83: 33, 26: 34, 73: 35, 57: 36, 36: 37, 41: 38, 37: 39, 8: 40, 19: 41, 2: 42, 40: 43, 16: 44, 56: 45, 75: 46, 10: 47, 39: 48, 7: 49, 49: 50, 67: 51, 32: 52, 4: 53, 71: 54, 97: 55, 54: 56, 48: 57, 42: 58, 80: 59, 79: 60, 34: 61, 58: 62, 51: 63, 76: 64, 89: 65, 47: 66, 9: 67, 28: 68, 1: 69, 25: 70, 29: 71, 94: 72, 44: 73, 62: 74, 22: 75, 74: 76, 95: 77, 86: 78, 61: 79, 70: 80, 77: 81, 96: 82, 85: 83, 14: 84, 12: 85, 15: 86, 98: 87, 92: 88, 93: 89, 13: 90, 88: 91, 30: 92, 65: 93, 90: 94, 55: 95, 63: 96, 23: 97, 21: 98, 82: 99, 91: 100}
